<a href="https://colab.research.google.com/github/Kengo-Akechi/Carbon-Emission/blob/main/ML_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Uploading the Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
file_path = '/content/drive/MyDrive/fossilfund_dataset.csv'
df = pd.read_csv(file_path)


In [ ]:
df.head()

,Fund profile: Shareclass name,Fund profile: Ticker,Fund profile: Fund name,Fund profile: Asset manager,Fund profile: Shareclass type,Fund profile: Shareclass inception date,Fund profile: Category group,Fund profile: Sustainability mandate,Fund profile: US-SIF member,Fund profile: Oldest shareclass inception date,...,"Prison Free Funds: Border industry, higher risk, weight","Prison Free Funds: Border industry, higher risk, asset","Prison Free Funds: Private prison operators, count","Prison Free Funds: Private prison operators, weight","Prison Free Funds: Private prison operators, asset",Gender Equality Funds: Gender equality score - Overall score (out of 100 points),Gender Equality Funds: Gender equality score - Gender balance in leadership and workforce (out of 40 points),Gender Equality Funds: Gender equality score - Equal compensation and work life balance (out of 30 points),Gender Equality Funds: Gender equality score - Policies promoting gender equality (out of 20 points),"Gender Equality Funds: Gender equality score - Commitment, transparency, and accountability (out of 10 points)"
0,1290 SmartBeta Equity A,TNBRX,1290 SmartBeta Equity Fund,1290 Funds,Open-end mutual fund,2014-11-12,International Equity Funds,Y,NaN,2014-11-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1290 SmartBeta Equity I,TNBRX,1290 SmartBeta Equity Fund,1290 Funds,Open-end mutual fund,2014-11-12,International Equity Funds,Y,NaN,2014-11-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1290 SmartBeta Equity R,TNBRX,1290 SmartBeta Equity Fund,1290 Funds,Open-end mutual fund,2014-11-12,International Equity Funds,Y,NaN,2014-11-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1290 SmartBeta Equity T,TNBRX,1290 SmartBeta Equity Fund,1290 Funds,Open-end mutual fund,2014-11-12,International Equity Funds,Y,NaN,2014-11-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13D Activist A,DDDCX,13D Activist Fund,13D Activist Fund,Open-end mutual fund,2011-12-28,U.S. Equity Fund,Y,NaN,2011-12-28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Importing libraries**

In [ ]:
#importing dependencies
import numpy as np
import re

from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import seaborn as sns

import string
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df.shape


(110510, 121)

**PREPROCESSING**

In [ ]:
# List of columns to keep
columns_to_keep = [
    'Fossil Free Funds: Relative carbon footprint (tonnes CO2 / $1M USD invested)',
    'Fossil Free Funds: Relative carbon intensity (tonnes CO2 / $1M USD revenue)',
    'Fossil Free Funds: Total financed emissions scope 1 + 2 (tCO2e)',
    'Fossil Free Funds: Total financed emissions scope 1 + 2 + 3 (tCO2e)',
    'Fossil Free Funds: Carbon footprint portfolio coverage by market value weight',
    'Fossil Free Funds: Carbon footprint portfolio coverage by number of disclosing titles',
    'Fossil Free Funds: Fossil fuel holdings, count',
    'Fossil Free Funds: Fossil fuel holdings, weight',
    'Deforestation Free Funds: Deforestation-risk producer, count',
    'Deforestation Free Funds: Deforestation-risk producer, weight',
    'Gender Equality Funds: Gender equality score (out of 100 points)',
    'Gender Equality Funds: Gender equality score, gender balance (out of 100 points)',
    'Gender Equality Funds: Gender equality score, gender policies (out of 100 points)',
    'Fund profile: Shareclass type',
    'Fund profile: Fund net assets',
    'Fund profile: Percent rated',
    'Fund profile: Shareclass inception date',
    'Fossil Free Funds: Fossil fuel grade',
    'Deforestation Free Funds: Deforestation grade',
    'Gender Equality Funds: Gender equality grade',
    'Fund profile: Asset manager',
    'Fund profile: Category group',
    'Fund profile: Sustainability mandate',
    'Fund profile: US-SIF member'
]

# Keeping only the selected columns
df_subset = df[columns_to_keep]

# Save the subset to a new CSV file or use it for further analysis
df_subset.to_csv('subset_dataset.csv', index=False)

In [ ]:
print(df_subset.dtypes)

Fossil Free Funds: Relative carbon footprint (tonnes CO2 / $1M USD invested)             float64
Fossil Free Funds: Relative carbon intensity (tonnes CO2 / $1M USD revenue)              float64
Fossil Free Funds: Total financed emissions scope 1 + 2 (tCO2e)                          float64
Fossil Free Funds: Total financed emissions scope 1 + 2 + 3 (tCO2e)                      float64
Fossil Free Funds: Carbon footprint portfolio coverage by market value weight            float64
Fossil Free Funds: Carbon footprint portfolio coverage by number of disclosing titles    float64
Fossil Free Funds: Fossil fuel holdings, count                                             int64
Fossil Free Funds: Fossil fuel holdings, weight                                          float64
Deforestation Free Funds: Deforestation-risk producer, count                               int64
Deforestation Free Funds: Deforestation-risk producer, weight                            float64
Gender Equality Funds: Gender 

**Handling Missing values**


In [ ]:
# Number of Nan values columnwise
for col in df_subset.columns:
    nan_count = df_subset[col].isna().sum()
    print(f"Number of NaN values in '{col}': {nan_count}")

Number of NaN values in 'Fossil Free Funds: Relative carbon footprint (tonnes CO2 / $1M USD invested)': 0
Number of NaN values in 'Fossil Free Funds: Relative carbon intensity (tonnes CO2 / $1M USD revenue)': 0
Number of NaN values in 'Fossil Free Funds: Total financed emissions scope 1 + 2 (tCO2e)': 0
Number of NaN values in 'Fossil Free Funds: Total financed emissions scope 1 + 2 + 3 (tCO2e)': 0
Number of NaN values in 'Fossil Free Funds: Carbon footprint portfolio coverage by market value weight': 0
Number of NaN values in 'Fossil Free Funds: Carbon footprint portfolio coverage by number of disclosing titles': 0
Number of NaN values in 'Fossil Free Funds: Fossil fuel holdings, count': 0
Number of NaN values in 'Fossil Free Funds: Fossil fuel holdings, weight': 0
Number of NaN values in 'Deforestation Free Funds: Deforestation-risk producer, count': 0
Number of NaN values in 'Deforestation Free Funds: Deforestation-risk producer, weight': 0
Number of NaN values in 'Gender Equality Fu

In [ ]:
df_subset.drop(columns=['Fund profile: US-SIF member'], inplace=True)
df_subset.drop(columns=['Fund profile: Sustainability mandate'], inplace=True)

KeyError: "['Fund profile: US-SIF member'] not found in axis"

In [ ]:
df['Gender Equality Funds: Gender equality grade'].fillna('Unknown', inplace=True)

In [ ]:
# Replace NaN values with mean value
mean_value = df_subset['Gender Equality Funds: Gender equality score (out of 100 points)'].mean()
df_subset['Gender Equality Funds: Gender equality score (out of 100 points)'].fillna(mean_value, inplace=True)


In [ ]:
# Replace NaN values with mean value
mean_value = df_subset['Gender Equality Funds: Gender equality score, gender balance (out of 100 points)'].mean()
df_subset['Gender Equality Funds: Gender equality score, gender balance (out of 100 points)'].fillna(mean_value, inplace=True)


In [ ]:
# Replace NaN values with mean value
mean_value = df_subset['Gender Equality Funds: Gender equality score, gender policies (out of 100 points)'].mean()
df_subset['Gender Equality Funds: Gender equality score, gender policies (out of 100 points)'].fillna(mean_value, inplace=True)

In [ ]:
column_with_nan ='Fossil Free Funds: Fossil fuel grade'
df_subset = df_subset.dropna(subset=[column_with_nan])

**Making target and features**


In [ ]:
columns = [
    'Fossil Free Funds: Relative carbon intensity (tonnes CO2 / $1M USD revenue)',
    'Fossil Free Funds: Total financed emissions scope 1 + 2 (tCO2e)',
    'Fossil Free Funds: Total financed emissions scope 1 + 2 + 3 (tCO2e)',
    'Fossil Free Funds: Carbon footprint portfolio coverage by market value weight',
    'Fossil Free Funds: Carbon footprint portfolio coverage by number of disclosing titles',
    'Fossil Free Funds: Fossil fuel holdings, count',
    'Fossil Free Funds: Fossil fuel holdings, weight',
    'Deforestation Free Funds: Deforestation-risk producer, count',
    'Deforestation Free Funds: Deforestation-risk producer, weight',
    'Fund profile: Shareclass type',
    'Fund profile: Fund net assets',
    'Fund profile: Percent rated',
    'Fund profile: Shareclass inception date',
    'Deforestation Free Funds: Deforestation grade',
    'Gender Equality Funds: Gender equality grade',
    'Fund profile: Asset manager',
    'Fund profile: Category group',
    'Fossil Free Funds: Relative carbon footprint (tonnes CO2 / $1M USD invested)',
    'Fossil Free Funds: Total financed emissions scope 1 + 2 (tCO2e)',
    'Fossil Free Funds: Total financed emissions scope 1 + 2 + 3 (tCO2e)'
]

# Selecting features (inputs)
features = df_subset[columns[:-8]]  # Selecting all columns except the last three, which are potential targets

# Selecting target variable (output)
target = df_subset['Fossil Free Funds: Relative carbon footprint (tonnes CO2 / $1M USD invested)']  # Choose appropriate target column

In [ ]:
features.head()

,Fossil Free Funds: Relative carbon intensity (tonnes CO2 / $1M USD revenue),Fossil Free Funds: Total financed emissions scope 1 + 2 (tCO2e),Fossil Free Funds: Total financed emissions scope 1 + 2 + 3 (tCO2e),Fossil Free Funds: Carbon footprint portfolio coverage by market value weight,Fossil Free Funds: Carbon footprint portfolio coverage by number of disclosing titles,"Fossil Free Funds: Fossil fuel holdings, count","Fossil Free Funds: Fossil fuel holdings, weight","Deforestation Free Funds: Deforestation-risk producer, count","Deforestation Free Funds: Deforestation-risk producer, weight",Fund profile: Shareclass type,Fund profile: Fund net assets,Fund profile: Percent rated
0,104.21,1846.27,5965.91,0.981044,0.844560,29,0.058496,5,0.005136,Open-end mutual fund,32236053,0.981064
1,104.21,1846.27,5965.91,0.981044,0.844560,29,0.058496,5,0.005136,Open-end mutual fund,32236053,0.981064
2,104.21,1846.27,5965.91,0.981044,0.844560,29,0.058496,5,0.005136,Open-end mutual fund,32236053,0.981064
3,104.21,1846.27,5965.91,0.981044,0.844560,29,0.058496,5,0.005136,Open-end mutual fund,32236053,0.981064
4,63.08,25580.25,104767.63,0.806359,0.333333,1,0.037922,0,0.000000,Open-end mutual fund,346536813,0.806360


In [ ]:
# Check data types of columns
print(features.dtypes)

Fossil Free Funds: Relative carbon intensity (tonnes CO2 / $1M USD revenue)              float64
Fossil Free Funds: Total financed emissions scope 1 + 2 (tCO2e)                          float64
Fossil Free Funds: Total financed emissions scope 1 + 2 + 3 (tCO2e)                      float64
Fossil Free Funds: Carbon footprint portfolio coverage by market value weight            float64
Fossil Free Funds: Carbon footprint portfolio coverage by number of disclosing titles    float64
Fossil Free Funds: Fossil fuel holdings, count                                             int64
Fossil Free Funds: Fossil fuel holdings, weight                                          float64
Deforestation Free Funds: Deforestation-risk producer, count                               int64
Deforestation Free Funds: Deforestation-risk producer, weight                            float64
Fund profile: Fund net assets                                                              int64
Fund profile: Percent rated   

In [ ]:
del_cols = ['Fund profile: Shareclass type']
features = features.drop(columns=del_cols, axis=1)


In [ ]:
'Fossil Free Funds: Relative carbon footprint (tonnes CO2 / $1M USD invested)' in df_subset.columns
'Fossil Free Funds: Relative carbon intensity (tonnes CO2 / $1M USD revenue)' in df_subset.columns

True

In [ ]:
target.head()

0    56.19
1    56.19
2    56.19
3    56.19
4    59.52
Name: Fossil Free Funds: Relative carbon footprint (tonnes CO2 / $1M USD invested), dtype: float64

**SPLITTING DATASET**

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

***TRAINING MODELS***

LINEAR REGRESSION MODEL

In [ ]:
from sklearn.linear_model import LinearRegression

# Initialize and train Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Make predictions on test set
y_pred = model.predict(X_test)

# Evaluate model performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')


Mean Squared Error: 4544.854191530387
R-squared: 0.7685474913724896


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Make predictions on test set
y_pred = model.predict(X_test)

# Calculate R-squared (R2) score using model.score
r2 = model.score(X_test, y_test)

print(f'R-squared (R2) Score using model.score: {r2}')


R-squared (R2) Score using model.score: 0.7685474913724896


In [ ]:
features.shape

(110505, 11)

**Using Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# Initialize Random Forest Regressor with desired hyperparameters
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model using the training data
rf_model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [ ]:
# Make predictions on test set using the trained model
rf_y_pred = rf_model.predict(X_test)

# Calculate R-squared (R2) score for Random Forest model
rf_r2 = rf_model.score(X_test, y_test)

print(f'R-squared (R2) Score for Random Forest Model: {rf_r2}')


R-squared (R2) Score for Random Forest Model: 0.9922454533299528


**Analysis of models**

Linear Regression


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Assuming 'features' and 'target' are your feature and target variables

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Initialize and train Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Cross-validation
cv_scores = cross_val_score(model, features, target, cv=5)  # 5-fold cross-validation
print("Cross-Validation Scores:", cv_scores)
print("Mean Cross-Validation Score:", np.mean(cv_scores))

# Make predictions on the test set
predictions = model.predict(X_test)

# Calculate evaluation metrics
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mse)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)


Cross-Validation Scores: [0.75335639 0.7641953  0.76511516 0.78857379 0.77647992]
Mean Cross-Validation Score: 0.7695441120120703
Mean Squared Error (MSE): 4544.854191530387
Mean Absolute Error (MAE): 38.578335426459006
Root Mean Squared Error (RMSE): 67.41553375543641


Random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Assuming 'features' and 'target' are your feature and target variables

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Initialize and train Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)  # Adjust hyperparameters as needed
rf_model.fit(X_train, y_train)

# Cross-validation
# cv_scores_rf = cross_val_score(rf_model, features, target, cv=5)  # 5-fold cross-validation
# print("Cross-Validation Scores (Random Forest):", cv_scores_rf)
# print("Mean Cross-Validation Score (Random Forest):", np.mean(cv_scores_rf))

# Make predictions on the test set
predictions_rf = rf_model.predict(X_test)

# Calculate evaluation metrics
mse_rf = mean_squared_error(y_test, predictions_rf)
mae_rf = mean_absolute_error(y_test, predictions_rf)
rmse_rf = np.sqrt(mse_rf)

print("Mean Squared Error (MSE) - Random Forest:", mse_rf)
print("Mean Absolute Error (MAE) - Random Forest:", mae_rf)
print("Root Mean Squared Error (RMSE) - Random Forest:", rmse_rf)


Mean Squared Error (MSE) - Random Forest: 152.2700451413163
Mean Absolute Error (MAE) - Random Forest: 2.593131021220802
Root Mean Squared Error (RMSE) - Random Forest: 12.339774922636


**NN**

In [ ]:
!pip install tensorflow


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Load your dataset and preprocess if needed
# Assuming 'features' and 'target' are your feature and target variables

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Standardize the features (optional but recommended for neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the neural network model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[X_train_scaled.shape[1]]),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Output layer with 1 neuron for regression task
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model on test data
predictions = model.predict(X_test_scaled)

# Calculate evaluation metrics
mse_nn = mean_squared_error(y_test, predictions)
mae_nn = mean_absolute_error(y_test, predictions)
rmse_nn = np.sqrt(mse_nn)

print("Mean Squared Error (MSE) - Neural Network:", mse_nn)
print("Mean Absolute Error (MAE) - Neural Network:", mae_nn)
print("Root Mean Squared Error (RMSE) - Neural Network:", rmse_nn)


Epoch 1/100
2211/2211 [==============================] - 8s 3ms/step - loss: 7248.8228 - val_loss: 4007.5398
Epoch 2/100
2211/2211 [==============================] - 10s 5ms/step - loss: 3641.3267 - val_loss: 3645.2661
Epoch 3/100
2211/2211 [==============================] - 14s 6ms/step - loss: 3470.8152 - val_loss: 3558.9319
Epoch 4/100
2211/2211 [==============================] - 10s 4ms/step - loss: 3366.9543 - val_loss: 3426.1782
Epoch 5/100
2211/2211 [==============================] - 5s 2ms/step - loss: 3284.2649 - val_loss: 3338.7349
Epoch 6/100
2211/2211 [==============================] - 6s 3ms/step - loss: 3214.9604 - val_loss: 3294.6719
Epoch 7/100
2211/2211 [==============================] - 5s 2ms/step - loss: 3148.6069 - val_loss: 3304.0950
Epoch 8/100
2211/2211 [==============================] - 6s 3ms/step - loss: 3096.2317 - val_loss: 3173.8403
Epoch 9/100
2211/2211 [==============================] - 7s 3ms/step - loss: 3048.1631 - val_loss: 3089.9314
Epoch 10/100
221